<a href="https://colab.research.google.com/github/anugoel890/chromadb/blob/main/Chromadb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
!pip install chromadb==0.3.21


In [63]:
!pip install pydantic==1.9.0

In [64]:
import chromadb
from chromadb.config import Settings

In [65]:
import pandas as pd

pdf = pd.read_csv("sample_data/labelled_newscatcher_dataset.csv", sep=";")
pdf["id"] = pdf.index
pdf_subset = pdf.head(1000)

In [66]:
chroma = chromadb.Client(
    Settings(
        chroma_db_impl="duckdb+parquet",
        persist_directory="sample_data"
    )
)


In [67]:
collection_name="collection_news"

In [68]:
chroma.list_collections()

[]

In [69]:
if len(chroma.list_collections()) > 0 and collection_name in [chroma.list_collections()[0].name]:
  chroma.delete_collection(name = collection_name)

In [70]:
metadata = [{"topic":x} for x in pdf_subset["topic"][:100]]
ids = [f"id{x}" for x in range(100)]

In [71]:
ids

['id0',
 'id1',
 'id2',
 'id3',
 'id4',
 'id5',
 'id6',
 'id7',
 'id8',
 'id9',
 'id10',
 'id11',
 'id12',
 'id13',
 'id14',
 'id15',
 'id16',
 'id17',
 'id18',
 'id19',
 'id20',
 'id21',
 'id22',
 'id23',
 'id24',
 'id25',
 'id26',
 'id27',
 'id28',
 'id29',
 'id30',
 'id31',
 'id32',
 'id33',
 'id34',
 'id35',
 'id36',
 'id37',
 'id38',
 'id39',
 'id40',
 'id41',
 'id42',
 'id43',
 'id44',
 'id45',
 'id46',
 'id47',
 'id48',
 'id49',
 'id50',
 'id51',
 'id52',
 'id53',
 'id54',
 'id55',
 'id56',
 'id57',
 'id58',
 'id59',
 'id60',
 'id61',
 'id62',
 'id63',
 'id64',
 'id65',
 'id66',
 'id67',
 'id68',
 'id69',
 'id70',
 'id71',
 'id72',
 'id73',
 'id74',
 'id75',
 'id76',
 'id77',
 'id78',
 'id79',
 'id80',
 'id81',
 'id82',
 'id83',
 'id84',
 'id85',
 'id86',
 'id87',
 'id88',
 'id89',
 'id90',
 'id91',
 'id92',
 'id93',
 'id94',
 'id95',
 'id96',
 'id97',
 'id98',
 'id99']

In [72]:
collection = chroma.create_collection(name=collection_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [73]:
collection.add(
    documents = pdf_subset["title"][:100].tolist(),
    metadatas = metadata,
    ids = ids,
)

In [74]:
results = collection.query(query_texts=["animal"],n_results=4)
results

{'ids': [['id99', 'id16', 'id1', 'id26']],
 'embeddings': None,
 'documents': [['Ghostwire: Tokyo confirms dog petting',
   'Tradeoff between the eyes and nose helps flies find their niche',
   'An irresistible scent makes locusts swarm, study finds',
   '‘It came alive:’ NASA astronauts describe experiencing splashdown in SpaceX Dragon']],
 'metadatas': [[{'topic': 'TECHNOLOGY'},
   {'topic': 'SCIENCE'},
   {'topic': 'SCIENCE'},
   {'topic': 'SCIENCE'}]],
 'distances': [[1.3118830919265747,
   1.5922542810440063,
   1.5983850955963135,
   1.635123372077942]]}

In [75]:
import json

In [76]:
print(json.dumps(results, indent=4))

{
    "ids": [
        [
            "id99",
            "id16",
            "id1",
            "id26"
        ]
    ],
    "embeddings": null,
    "documents": [
        [
            "Ghostwire: Tokyo confirms dog petting",
            "Tradeoff between the eyes and nose helps flies find their niche",
            "An irresistible scent makes locusts swarm, study finds",
            "\u2018It came alive:\u2019 NASA astronauts describe experiencing splashdown in SpaceX Dragon"
        ]
    ],
    "metadatas": [
        [
            {
                "topic": "TECHNOLOGY"
            },
            {
                "topic": "SCIENCE"
            },
            {
                "topic": "SCIENCE"
            },
            {
                "topic": "SCIENCE"
            }
        ]
    ],
    "distances": [
        [
            1.3118830919265747,
            1.5922542810440063,
            1.5983850955963135,
            1.635123372077942
        ]
    ]
}


In [77]:
collection.query(query_texts=["animal"], where={"topic": "SCIENCE"}, n_results=10)

{'ids': [['id16',
   'id1',
   'id26',
   'id36',
   'id19',
   'id18',
   'id27',
   'id20',
   'id2',
   'id38']],
 'embeddings': None,
 'documents': [['Tradeoff between the eyes and nose helps flies find their niche',
   'An irresistible scent makes locusts swarm, study finds',
   '‘It came alive:’ NASA astronauts describe experiencing splashdown in SpaceX Dragon',
   'Cause Of Massive Fish Kill In Shinnecock Canal Not Clear - 27 East',
   'Nasa SpaceX crew return: Dragon capsule splashes down',
   'In rare find, fossil shows how Cretaceous-era ‘hell ant’ ate its prey with weird jaws',
   'When did humans find out how to use fire?',
   'Take a look at Mar’s eerie nightglow',
   'Artificial intelligence warning: AI will know us better than we know ourselves',
   "'Zombie cicadas' under the influence of a mind controlling fungus have returned to West Virginia"]],
 'metadatas': [[{'topic': 'SCIENCE'},
   {'topic': 'SCIENCE'},
   {'topic': 'SCIENCE'},
   {'topic': 'SCIENCE'},
   {'topic

In [78]:
collection.get(
    ids=["id2"],
)

{'ids': ['id2'],
 'embeddings': None,
 'documents': ['Artificial intelligence warning: AI will know us better than we know ourselves'],
 'metadatas': [{'topic': 'SCIENCE'}]}

In [79]:
collection.update(
    ids=["id2"],
    metadatas=[{"topic": "TECHNOLOGY"}],
)


In [80]:
collection.get(
    ids=["id2"],
)

{'ids': ['id2'],
 'embeddings': None,
 'documents': ['Artificial intelligence warning: AI will know us better than we know ourselves'],
 'metadatas': [{'topic': 'TECHNOLOGY'}]}

In [81]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
lm_model = AutoModelForCausalLM.from_pretrained(model_id)

pipe = pipeline(
    "text-generation",
    model=lm_model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    device_map="auto",
)


In [84]:
results["documents"][0]

['Ghostwire: Tokyo confirms dog petting',
 'Tradeoff between the eyes and nose helps flies find their niche',
 'An irresistible scent makes locusts swarm, study finds',
 '‘It came alive:’ NASA astronauts describe experiencing splashdown in SpaceX Dragon']

In [82]:
context = " ".join([f"#{str(i)}" for i in results["documents"][0]])
context

'#Ghostwire: Tokyo confirms dog petting #Tradeoff between the eyes and nose helps flies find their niche #An irresistible scent makes locusts swarm, study finds #‘It came alive:’ NASA astronauts describe experiencing splashdown in SpaceX Dragon'

In [86]:
question = "What's the latest news on space development?"

prompt_template = f"Relevant context: {context}\n\n The user's question: {question}"


In [87]:
lm_response = pipe(prompt_template)
print(lm_response[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Relevant context: #Ghostwire: Tokyo confirms dog petting #Tradeoff between the eyes and nose helps flies find their niche #An irresistible scent makes locusts swarm, study finds #‘It came alive:’ NASA astronauts describe experiencing splashdown in SpaceX Dragon

 The user's question: What's the latest news on space development?


SpaceX announced early this morning that it is relocating its Falcon 9 rocket and first flight to a hangar outside of New York City. Crews are expected to begin flying to the pad by the end of June, and SpaceX is also aiming to begin building two Falcon 9s to launch from the launch pad during the first stage's maiden flight in March.


The rocket will launch at speeds not seen since the Falcon 9 was ordered to the International Space Station in 1986. Though it has had trouble performing well in space so far, the Falcon 9 is an ideal launch vehicle for SpaceX's plan to get to the International Space Station in May.


NASA's robotic robotic arm of the ISS will h